# Example of working with data visualization and a simple dashboard

In [1]:
from dash import Dash, dcc, html
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to the PostgreSQL server with data and import the data

In [4]:
# Connection settings
DB_USER = 'intern'
DB_PASSWORD = 'intern12345678'
DB_HOST = 'localhost'
DB_PORT = '5432'  # Replace with your port if modified
DB_NAME = 'sample_superstore'


In [6]:
# Database connection
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)


In [8]:
query = "SELECT * FROM orders;"

In [10]:
query_people = "SELECT * FROM people;"

In [12]:
# Download data into pandas DtaFarame
df = pd.read_sql(query, conn)

/tmp/ipykernel_14740/3869087620.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [14]:
# Download data into pandas DtaFarame
df_people = pd.read_sql(query_people, conn)

/tmp/ipykernel_14740/2856199825.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_people = pd.read_sql(query_people, conn)


In [16]:
# Check data
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2018-152156,43412,43415,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2018-152156,43412,43415,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2018-138688,43263,43267,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2017-108966,43019,43026,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2017-108966,43019,43026,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [18]:
df_people

,Person,Region
0,Anna Andreadi,West
1,Chuck Magee,East
2,Kelly Williams,Central
3,Cassandra Brandow,South


### Date and time were not imported correctly

In [21]:
df['Order Date'] = pd.to_datetime(df['Order Date'], origin='1899-12-30', unit='D')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], origin='1899-12-30', unit='D')


In [23]:
df.info()  # check data types in Order Date and Ship Date columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   object        
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

# Create pivot tables for the dashboard

In [26]:
# Group data by date, summing sales and profit, and calculating the average discount
df_grouped = df.groupby('Order Date').agg({
    'Sales': 'sum', 
    'Profit': 'sum',
    'Discount': 'mean'  # Add calculation of average discount
}).reset_index()

# Add Profit Ratio column
df_grouped['Profit Ratio'] = (df_grouped['Profit'] / df_grouped['Sales']) * 100

# Add Month column
df['Month'] = df['Order Date'].dt.to_period('M')

# Group data by month and segment
df_segment_grouped = df.groupby(['Order Date', 'Segment']).agg({'Sales': 'sum'}).reset_index()

# Merge tables on the "Region" column
df = df.merge(df_people, on='Region', how='left')
# Group data by month and segment for managers
df_grouped_by_manager = df.groupby(['Order Date', 'Person']).agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()

# Design a dashboard for manager sales and profit trends

In [29]:
# Create a Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Sales and Profit Analysis"),

    # Date range filter
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=df_grouped['Order Date'].min(),
        end_date=df_grouped['Order Date'].max(),
        display_format='YYYY-MM-DD'
    ),
    
    # Graph to display sales and profit trends
    dcc.Graph(id='sales-profit-graph'),

    # Graph to display Profit Ratio and average discount trends
    html.Div([
        html.Div([
            html.H2("Profit Ratio Trend"),
            html.H3(id='avg-discount', style={'display': 'inline-block', 'margin-left': '20px'})
        ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'space-between'}),
        dcc.Graph(id='profit-ratio-graph')
    ], style={'margin': '20px'}),

    # New graph for segment sales trend
    html.Div([
        html.H2("Segment Sales Trend"),
        dcc.Graph(id='segment-sales-graph')
    ], style={'margin': '20px'}),

    # New graph for manager sales and profit trend
    html.Div([
        html.H2("Manager Sales and Profit Trend"),
        dcc.Graph(id='manager-sales-profit-graph')
    ], style={'margin': '20px'})
])

# Callback to update graphs and average discount
@app.callback(
    [Output('sales-profit-graph', 'figure'),
     Output('profit-ratio-graph', 'figure'),
     Output('segment-sales-graph', 'figure'),
     Output('manager-sales-profit-graph', 'figure'),
     Output('avg-discount', 'children')],
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_graphs(start_date, end_date):
    # Filter data based on the selected date range
    filtered_df = df_grouped[(df_grouped['Order Date'] >= start_date) & (df_grouped['Order Date'] <= end_date)]
    
    # Sales and profit trend graph
    fig_sales_profit = px.line(
        filtered_df, x='Order Date', y=['Sales', 'Profit'],
        title='Sales and Profit Trend'
    )
    fig_sales_profit.update_layout(xaxis_title="Date", yaxis_title="Amount", legend_title="Metrics")

    # Profit Ratio trend graph
    fig_profit_ratio = px.line(
        filtered_df, x='Order Date', y='Profit Ratio',
        title='Profit Ratio Trend'
    )
    fig_profit_ratio.update_layout(xaxis_title="Date", yaxis_title="Profit Ratio (%)")

    # Segment sales trend graph
    filtered_df_segment = df_segment_grouped[(df_segment_grouped['Order Date'] >= start_date) & (df_segment_grouped['Order Date'] <= end_date)]
    fig_segment_sales = px.line(
        filtered_df_segment, x='Order Date', y='Sales', color='Segment',
        title='Segment Sales Trend'
    )
    fig_segment_sales.update_layout(xaxis_title="Date", yaxis_title="Sales", legend_title="Segment")

    # Filter manager data based on the selected date range
    filtered_df_manager = df_grouped_by_manager[(df_grouped_by_manager['Order Date'] >= start_date) & (df_grouped_by_manager['Order Date'] <= end_date)]
    
    # Manager sales and profit trend graph
    fig_manager_sales_profit = px.bar(
        filtered_df_manager, x='Order Date', y=['Sales', 'Profit'], color='Person',
        title='Manager Sales and Profit Trend', barmode='group'
    )
    fig_manager_sales_profit.update_layout(xaxis_title="Date", yaxis_title="Amount", legend_title="Manager")

    # Check if the Discount column exists before calculating the average discount
    avg_discount = "No discount data available"  # default value
    if 'Discount' in filtered_df.columns:
        avg_discount = f"Average Discount (Avg. Discount): {filtered_df['Discount'].mean():.2f}%"
    
    return fig_sales_profit, fig_profit_ratio, fig_segment_sales, fig_manager_sales_profit, avg_discount

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True)